In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import lightgbm as lgb
import xgboost as xgb
import warnings
import yfinance as yf 
import pandas as pd

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")


ticker="AAPL"
    
    #나스닥 파일 불러오기
data_path = 'data/nasdq_20241104.csv'
data = pd.read_csv(data_path)

    
start_date = '2010-01-04'
end_date = '2024-10-25'
stock_data = yf.download(ticker, start=start_date, end=end_date)
    
    #AAPL 데이터 전처리
stock_data.reset_index(inplace=True)
stock_data['Date'] = stock_data['Date'].dt.strftime('%Y-%m-%d')
stock_data.columns = ['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
stock_data.drop(columns=['Adj Close'], inplace=True)
    
    #Nasdaq 데이터 전처리
#global_data = data.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume'])

    #AAPL, Nasdaq merge
stock_merge_data = pd.merge(stock_data, data, on='Date', how='inner')

    #merge data 전처리
stock_merge_data['Date'] = pd.to_datetime(stock_merge_data['Date'])
stock_merge_data.set_index('Date', inplace=True)
stock_merge_data.columns = stock_merge_data.columns.str.replace(' ', '')

stock_merge_data['Close_InterestRate_Corr'] = stock_merge_data['Close'].rolling(252).corr(stock_merge_data['InterestRate'])
stock_merge_data['Close_VIX_Corr'] = stock_merge_data['Close'].rolling(252).corr(stock_merge_data['VIX'])
stock_merge_data['Rolling_Volatility'] = stock_merge_data['Close'].rolling(window=30).std()

stock_merge_data['Daily_Return'] = stock_merge_data['Close'].pct_change()
stock_merge_data['Volatility'] = stock_merge_data['Close'].rolling(window=30).std() # 이동 표준 편차를 계산하라
stock_merge_data['Rolling_Mean_Close'] = stock_merge_data['Close'].rolling(window=30).mean()
stock_merge_data.dropna(inplace=True)
 

X = stock_merge_data.drop(['Close', 'High', 'Low', 'Volume'], axis=1)  # Ensure 'Close' is dropped to create the feature set
y = stock_merge_data['Close']  # Target variable is 'Close' price

    # Step 1: Replace infinite values with NaN
X.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Step 2: Check for NaN values and handle them
    # Using forward fill to handle NaN values (you can adjust this as needed)
X.fillna(method='ffill', inplace=True)



    # Step 3: Standardize the data
#scaler = StandardScaler()
#scaler.fit(X)
#X_scaled = scaler.transform(X)



    # Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)




   # Initialize and train a random forest model
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf.predict(X_test)

aa = pd.DataFrame()
aa['Close'] = y_test
aa['Predict'] = y_pred_rf 
X

[*********************100%***********************]  1 of 1 completed


,Open,InterestRate,ExchangeRate,VIX,TEDSpread,EFFR,Gold,Oil,Close_InterestRate_Corr,Close_VIX_Corr,Rolling_Volatility,Daily_Return,Volatility,Rolling_Mean_Close
Date,,,,,,,,,,,,,,
2010-12-31,11.533929,0.18,1.32690,17.75,0.18,0.13,1421.099976,91.379997,0.642281,-0.182643,0.185331,-0.003399,0.185331,11.390191
2011-01-03,11.630000,0.17,1.33710,17.61,0.18,0.19,1422.599976,91.550003,0.633936,-0.190885,0.183933,0.021732,0.183933,11.415357
2011-01-04,11.872857,0.17,1.33060,17.38,0.16,0.18,1378.500000,89.379997,0.625355,-0.200087,0.177785,0.005219,0.177785,11.444595
2011-01-05,11.769643,0.17,1.31710,17.02,0.16,0.18,1373.400024,90.300003,0.615647,-0.210336,0.191963,0.008180,0.191963,11.469167
2011-01-06,11.954286,0.17,1.30270,17.40,0.15,0.17,1371.400024,88.379997,0.605810,-0.220130,0.190102,-0.000808,0.190102,11.498929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-18,236.179993,4.83,1.08557,18.03,0.09,4.83,2713.699951,69.220001,-0.506098,0.239333,4.744330,0.012277,4.744330,226.382333
2024-10-21,234.449997,4.83,1.08469,18.37,0.09,4.83,2723.100098,70.559998,-0.515898,0.254553,4.971251,0.006298,4.971251,226.901333
2024-10-22,233.889999,4.83,1.08055,18.20,0.09,4.83,2744.199951,72.089996,-0.524987,0.271290,5.060168,-0.002622,5.060168,227.426333


In [37]:
ser_rf=pd.Series(y_pred_rf)
ser_rf

0       28.570100
1       12.216068
2       15.255550
3       38.297674
4       28.834525
          ...    
686    140.377602
687     14.972096
688    125.005400
689    141.023901
690     11.934375
Length: 691, dtype: float64